In [47]:
import pandas as pd
import numpy as np

import plotly.express as px

import clip
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel


In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [14]:
train = pd.read_csv('../../data/train.csv')
test = pd.read_csv('../../data/test.csv')

In [16]:
train.head()

,id,description,object_img
0,520,Фотография. Г. Пермь. Здание горисполкома. ПО...,799
1,1817,Фотонегатив пленочный. Труппа театра «У моста»...,854
2,188,"Изразец гладкий расписной ""пермский""- карниз А...",1794
3,1472,Фотонегатив пленочный. Серебряная чаша из Хоре...,526
4,1664,Фотонегатив стеклянный. Этнография коми-пермяк...,244


In [69]:
train["object"] = train["description"].map(lambda x : x.split()[0].replace('.', '').replace(',', ''))

In [70]:
px.histogram(
    train["object"]
).show()

In [36]:
import gc

gc.collect()
with torch.no_grad():
    torch.cuda.empty_cache()

In [62]:
test_index = 0
image = train.object_img[test_index]
path_to_image = f'../data/train/{image}.png'

image = Image.open(path_to_image)
text = train['object'].unique().tolist()
text = ['Photography',
'Photonegative',
 'Tile',
'CCD',
'Etude',
'Photocopy',
 'Herbarium',
 'Coin',
'Photo Postcard',
 'Gun',
 'Slide',
 'Clock',
'Two',
'Painting',
'Sword',
 'Barns',
 'Flail',
'Rifle',
 'Gun',
'Cannon',
'Checker',
 'Grenade',
'Arrow',
 'Berdysh',
 'Cleaver',
'Friends',
'Projectile',
 'Blank',
 'Core',
'Dagger',
 'Four',
'Carcass',
 'Owl',
'Knife',
'Horse',
'Chair',
 'State-owned',
 'Alarm Clock',
'Spear',
'Harrowing',
 'Revolver',
 'Postcard',
 'Scimitar',
'Blunderbuss',
'Types',
'Bomb',
 'Transportation',
 'Butt',
'Bow',
 'Espadron',
'Axe',
 'Kokoshnik',
 'Halberd',
'Saber',
 'Peasants',
'Adaptation',
 'Peasant Women',
'Group',
'Falconet',
'Fitting',
'Bayonet',
 'Suspension',
 'Millstones',
 'Teenagers',
'Cartridge',
'Barrel',
 'Rohatina',
 'Wooden',
'Whip',
 'Svetets',
'Aerial bomb',
 'Squeaker',
'Album',
'Old Ladies',
 'Broadsword',
'Two',
'Mine',
'Plate']

inputs = processor(text=text, images=image, return_tensors="pt", padding=True).to(device)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  #

In [63]:
np.argmax(probs.cpu().detach().numpy())

8

In [64]:
text[8]

'Photo Postcard'

In [65]:
image.size

(344, 258)

In [68]:
train['object'].unique().tolist()

['Фотография',
 'Фотонегатив пленочный',
 'Изразец гладкий расписной "пермский"- карниз А/РУС-143 ПОКМ-18870/50 [эмаль (белая зеленая)]глина (красная)эмалированиеформовкароспись     Карниз с выпукло-вогнутой лицевой поверхностью и росписью в виде рамки зеленой эмали на белом фоне по краям лицевой пластины (ПГУ 1706/246 Р-2 уч',
 'Фотонегатив стеклянный',
 'ПЗС',
 'Этюд',
 'Фотокопия',
 'Гербарий',
 'Монета ДМ/Н-191 ПОКМ-17741/141 серебро; штамповка д',
 'Фотооткрытка',
 'Этюд [Коровы]',
 'Изразец гладкий расписной - поясовой  ПОКМ-19182/5 росписьформовкаэмаль (белая синяя)глина (красная грубокирпичная)     Изразец поясовой из красной грубокирпичной глины',
 'Пистолет однозарядный самодельный ОР-30 ПОКМ-4583 деревосталькустарное производство     Ствол откидной круглый в сечении без нарезов с  запирающей защелкой наверху (по типу револьверов Смита и Вессона)',
 'Диапозитив стеклянный',
 'Часы карманные механические с цифровой индексацией М-709 ПКМ-21149 заводское производство латунь нике